# CS572 Project
* We selected the paper: {}
* Based on the data and model of given paper, we are planning to make an extension of LSTM that predicts {}.

## Preparing

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from sklearn.model_selection import train_test_split
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

data_path = 'Data' # add path

input_size = 8
hidden_size = 128
num_layers = 3
num_epochs = 2000
batch_size = 32
learning_rate = 0.001
weight_decay=0.00001

Check if your computer is prepared to run pytorch model with CUDA.

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # x -> batch_size, seq, input_size
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
#         out, _ = self.rnn(x, h0)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        
        # Decode the hidden state of the last time step
        # out : batch_size, seq_length, hidden_size
        # out (N, 9, 128)  ???
        out = out[:, -1, :]
        out = self.fc(out)
        
        return out

Data Loading

In [4]:
data_set = []
for path_idx in os.listdir(data_path):
    cur_path = os.path.join(data_path, path_idx)
    print(cur_path)
    input_file = np.loadtxt(cur_path, dtype='float', delimiter=',')
    # cell = nn.RNN(input_size=4, hidden_size=2, batchfirst=True)
    inputs = torch.Tensor(input_file)
#     print(inputs[0])
    
    inputs = inputs[:len(inputs)-len(inputs)%40]
#     labels = inputs[:, [5]]
# #     print(labels.shape)
#     labels = labels.view([-1, 40, 1])
    
    inputs = inputs[:, [1,2,3,4,6,7,8,9,5]]
    inputs = inputs.view([-1,40,9])
    
#     for test a file
    data_set += [inputs]
    break
   
# print(data_set)
print("load complete")

Data\190722_tension_test_space_1_trial_1.csv
load complete


In [5]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=data_set[0],
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=data_set[0],
                                          batch_size=batch_size, 
                                          shuffle=False)
print("loader setting complete")

loader setting complete


In [6]:
# Training and Testing: To train fingertip forces from input
# data, a three-layer BT-LSTM with 128 hidden units is used.
# Sequence time length of BT-LSTM was set to T = 40. The final
# output of the BT-LSTM layer is fed into the final fully-connected
# layer to obtain the one-dimensional output. The mean squared
# error (MSE) function is used to measure the loss between the
# predicted and the ground truth contact force. During training,
# the number of epoch was 2000, and the mini batch size was 32.
# Adam optimizer was used with 0.001 learning rate and 10−5
# weight decay
lstm = LSTM(input_size, hidden_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate, weight_decay=weight_decay)

Train model

In [ ]:
for i, x in enumerate(train_loader):
#     print (i, x)
#     print(x.shape)
#     inputs, labels = torch.split(x, [8, 1], 2)
#     print(inputs.shape)
#     print(labels.shape)
    break

In [ ]:
for i in test_loader:
    print (i)

In [8]:
# Train the model
for epoch in range(num_epochs):
# for epoch in range(1):    
    for i, inputs in enumerate(train_loader):
        inputs, labels = torch.split(inputs, [8, 1], 2)
        inputs = inputs.to(device)
        labels = labels[:,[39],:].view([-1,1])
        labels = labels.to(device)
        
        # Forward pass
        outputs = lstm(inputs)
        
#         print(labels.shape)
        
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 10 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, loss.item()))

Epoch [10/2000], Loss: 1.2059
Epoch [20/2000], Loss: 0.5222
Epoch [30/2000], Loss: 0.7755
Epoch [40/2000], Loss: 1.7006
Epoch [50/2000], Loss: 1.6352
Epoch [60/2000], Loss: 0.4719
Epoch [70/2000], Loss: 0.3469
Epoch [80/2000], Loss: 0.6358
Epoch [90/2000], Loss: 1.0986
Epoch [100/2000], Loss: 0.7600
Epoch [110/2000], Loss: 1.3751
Epoch [120/2000], Loss: 0.9058
Epoch [130/2000], Loss: 0.7072
Epoch [140/2000], Loss: 0.9226
Epoch [150/2000], Loss: 0.7239
Epoch [160/2000], Loss: 1.9758
Epoch [170/2000], Loss: 0.4032
Epoch [180/2000], Loss: 0.5901
Epoch [190/2000], Loss: 0.7828
Epoch [200/2000], Loss: 0.7609
Epoch [210/2000], Loss: 0.6584
Epoch [220/2000], Loss: 0.7442
Epoch [230/2000], Loss: 0.5561
Epoch [240/2000], Loss: 0.9940
Epoch [250/2000], Loss: 0.6129
Epoch [260/2000], Loss: 0.6423
Epoch [270/2000], Loss: 1.3106
Epoch [280/2000], Loss: 0.5949
Epoch [290/2000], Loss: 1.1711
Epoch [300/2000], Loss: 0.9932
Epoch [310/2000], Loss: 0.6089
Epoch [320/2000], Loss: 0.3705
Epoch [330/2000],

Test model

In [11]:
lstm.eval()
with torch.no_grad():
    rmse = 0
    mae = 0
    for inputs in test_loader:
        inputs, labels = torch.split(inputs, [8, 1], 2)
        inputs = inputs.to(device)
        labels = labels[:,[39],:].view([-1,1])
        labels = labels.to(device)
        outputs = lstm(inputs)
        
        rmse += mean_squared_error(labels, outputs)
        mae += mean_absolute_error(labels, outputs)
        
    rmse = sqrt(rmse)

    print('Test Accuracy of the model RMSE: {} %'.format(rmse))
    print('Test Accuracy of the model MAE: {} %'.format(mae)) 

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

Save model

In [ ]:
torch.save(lstm.state_dict(), 'model.ckpt')